In [1]:
from Carcassonne_Game.Carcassonne import CarcassonneState
from TicTacToe_Game.TicTacToe import TicTacToeState 
from Function_Optimisation_Game.Function_Optimisation import FunctionOptimisationState
from player.Player import RandomPlayer
from player.MCTS_Player import MCTSPlayer
from player.MCTS_RAVE_Player import MCTS_RAVEPlayer
from player.MCTS_ES_BACK_Player import MCTS_ES_BACK_Player
from player.MCTS_ES_BACK_SEM_Player import MCTS_ES_BACK_SEM_Player

import Experimental_Setup as exps
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from plotly.tools import make_subplots

import os
import pandas as pd
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import numpy as np
import ast
import random
import math
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import bernoulli
from itertools import repeat

In [2]:
def show_search(data_list, function, title, divisions, n_buckets = 100, type="divisions", subplot_titles=None, max_x_location = None, y_ref_value = None):
   #print("received subplot_titles", subplot_titles)
   
   #print("edited subplot_titles", subplot_titles)
   if divisions in [2,3]: colors = ["#696969","#4f4f4f","#2f2f2f"]
   if divisions==4: colors = ["#cf0000","#a2000d","#740017","#53001b"]
   n_plots = len(data_list)
   even_spaces = 1/(n_plots+1)
   row_heights = [even_spaces for _ in range(n_plots)] + [even_spaces]
   fig = make_subplots(rows=n_plots+1, cols=1,shared_xaxes=True,vertical_spacing=0.03,row_heights=row_heights
                        ,subplot_titles = subplot_titles, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)])
   if function is not None:
      x = np.linspace(0.001,1,5000)
      y = [function([i]) for i in x]
      fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=n_plots+1,col=1)

   show_legend = [False] + [False for _ in range(n_plots)]
   for i,data in enumerate(data_list):
      for div in range(divisions):
         if div%divisions == 0: s1 = "{:2.0f}".format(100*(div/divisions))
         else: s1 = "{:2.1f}".format(100*(div/divisions))
         if (div+1)%divisions == 0: s2 = "{:2.0f}".format(100*((div+1)/divisions))
         else: s2 = "{:2.1f}".format(100*((div+1)/divisions))

         div_name = s1 + "% to " + s2 + "%"
         temp_data = data.loc[data["id_block"]==div]
         fig.add_trace(go.Histogram(x=temp_data.x, nbinsx=n_buckets, xbins={"start":0,"end":1,"size":1/n_buckets}
                  #, legendgroup=div, name=div_name
                  , showlegend=show_legend[i]
                  , marker={"color":colors[div]}),row=i+1,col=1)
         #fig.add_trace(go.Scatter(x=x, y=y, showlegend=False,marker={"color":"black"}),row=i+1,col=1,secondary_y=True)

   #fig.update_layout(barmode='stack')

   fig.update_layout(margin=dict(l=10, r=10, t=30, b=20),width=800,height=800,plot_bgcolor='rgba(0,0,0,0)',title={"text":title}
               ,barmode='stack'
               ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=-0.65,
                    xanchor="center",
                    x=0.5,  
                    font = dict(family = "Arial", size = 14, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    itemwidth = 30
                    )  )
   #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
   fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, showgrid=True, nticks=5, gridcolor="red", gridwidth=0.1)#, dtick=1000, showgrid=True, gridcolor="red", gridwidth=0.1)#, nticks=10, showgrid=True, griddash="dash", gridwidth=0.5, gridcolor="LightPink")
   #fig.update_layout(yaxis4=dict(dtick=500, showgrid=True, gridcolor="red", gridwidth=0.1))

   list_of_shapes = []
   if max_x_location is not None:
      for subplot_n in range(1,n_plots+2):
         yref = "y" + str(subplot_n*2)
         list_of_shapes.append({'type': 'line','y0':0,'y1': 1,'x0':max_x_location, 
                                    'x1':max_x_location,'xref':"x",'yref':yref,
                                    'line': {'color': 'red','width': 1}})
   if y_ref_value is not None:
         for subplot_n in range(1,n_plots+1):
            yref = "y" + str(subplot_n*2-1)
            list_of_shapes.append({'type': 'line','y0':y_ref_value,'y1': y_ref_value,'x0':0, 
                                       'x1':1,'xref':"x",'yref':yref,
                                       'line': {'color': 'red','width': 1, "dash":"dash"}})
   if list_of_shapes != []:
      fig['layout'].update(shapes=list_of_shapes)
      """
      
      for subplot_n in range(1, n_plots+2):
         yref = "y" + str(subplot_n)
         fig.add_shape(go.layout.Shape(type="line", yref=yref, xref="x", x0=max_x_location, x1 = max_x_location, y0=0, y1=1, line=dict(color="red", width=1),),row=subplot_n, col=1)
      """
   #fig.show()
   return fig

def get_subset(data, agent_name, f_index, c_param=None):
   temp_data = data.loc[data["f_index"]==f_index]
   if c_param is not None:
      temp_data = temp_data.loc[temp_data["c_param"]==c_param]
   #print(temp_data["player"].unique())
   temp_data = temp_data.loc[temp_data["player"] == agent_name]
   return temp_data

dummy_state = FunctionOptimisationState(players=[None], function=0, ranges=[[0,1]], splits=2)
functions = dummy_state.function_list
join = "/"
logs_path = "logs/FO"
output_name = "collective_tree_logs.csv"
data = pd.read_csv(logs_path + join + "collective_tree_logs.csv")
n_buckets = 500
f_max_locations = [0.5,0.867,None,0.1,0.1]

for f_index in [0,1,2,3,4]:
   agents_names = data["player"].unique()
   print(agents_names)
   c_params = data["c_param"].unique()
   c_params.sort()
   generic_name = "MCTS_c"
   agents_names = [x for x in agents_names]
   names = []
   data_list = []

   #order names
   for c in c_params:
      string_c = str(c)
      if string_c.split(".")[-1] == "0":
         string_c = string_c[:-2]
      names.append(generic_name + string_c)
   for name in agents_names:
      if "SE_MCTS" in name:
         names.append(name)
   for name in names:
      temp_data = get_subset(data, name, f_index)
      data_list.append(temp_data)
   
   treated_names = []
   for it,st in enumerate(names):
      new_string = st.replace("_c"," C = ")
      new_string = new_string.replace("1.414","sqrt(2)fred")
      new_string = new_string.split("fred")[0]
      if "SE_MCTS" in new_string:
         if "2600" in new_string:
            new_string = "SE_MCTS partial simulations"
         else:
            new_string = "SE_MCTS complete simulations"
      treated_names.append(new_string)
   
   
   plot = show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, type="divisions", subplot_titles = treated_names+["Function "+str(f_index+1)], max_x_location=f_max_locations[f_index], y_ref_value=None)
   #plot.write_image(os.path.join(logs_path, "f" + str(f_index) + '_v2.png'), width=800, height=800)
   #depth_plot = show_search_depth(data_list)
   #depth_plot.write_image(os.path.join(logs_path, "depth" + str(f_index) + '.png'), width=600, height=350)
   #plot = show_search2(data_list, functions[f_index], "", 3, n_buckets = 200, type="divisions", subplot_titles = agents_names+["Function "+str(f_index)])
   plot.show()
   #depth_plot.write_image(os.path.join(logs_path, "depth_f" + str(f_index) + '.png'), width=800, height=400)
   #plot.write_image(os.path.join(logs_path, "depth_histo_max_2k_f" + str(f_index) + '.png'), width=800, height=800)